In [69]:
!nvidia-smi

Sat Jan 25 09:40:08 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K20Xm         On   | 00000000:08:00.0 Off |                    0 |
| N/A   47C    P0    57W / 235W |   5485MiB /  5700MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K20Xm         On   | 00000000:84:00.0 Off |                    0 |
| N/A   52C    P0    56W / 235W |   5485MiB /  5700MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [2]:
go = False
if go:
    
    import os
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
    import tensorflow as tf
    from keras import backend as K

    set_gpu=True

    if set_gpu:
        os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";  # The GPU id to use, usually either "0" or "1";
        os.environ["CUDA_VISIBLE_DEVICES"]="0";  # Do other imports now...
        print(os.environ["CUDA_VISIBLE_DEVICES"])


        num_cores = 39

        GPU=True
        CPU=False

        if GPU:
            num_GPU = 1
            num_CPU = 1
        if CPU:
            num_CPU = 1
            num_GPU = 0

        config = tf.ConfigProto(intra_op_parallelism_threads=num_cores,
                                inter_op_parallelism_threads=num_cores, 
                                allow_soft_placement=True,
                                device_count = {'CPU' : num_CPU,
                                                'GPU' : num_GPU}
                               )

        session = tf.Session(config=config)
        K.set_session(session)


In [3]:
#!wget https://raw.githubusercontent.com/DBertazioli/host/master/datas/df_final_processed.csv

In [68]:
#!pip install pyforest
#from pyforest import *
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import keras

In [105]:
df_harmonics = pd.read_csv("../data/output/harmonics.csv", index_col=0)
df_harmonics.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9
1,1,0.192241,0.111166,0.017747,-0.012548,0.033027,-0.010471,0.014676,-0.007925
2,2,0.097356,0.057627,-0.001360,0.004587,0.072702,0.000261,0.014648,0.005893
3,3,-0.021915,-0.011354,-0.020139,0.020487,0.107396,0.010978,0.014621,0.019311
4,4,-0.135697,-0.077292,-0.034058,0.030868,0.134730,0.021047,0.014593,0.031421
5,5,-0.215488,-0.122519,-0.039756,0.032934,0.152834,0.029876,0.014565,0.041401


In [106]:
df = pd.read_csv("../data/df_final_processed.csv", index_col = 0)
df.head()

from keras.preprocessing.sequence import TimeseriesGenerator

#some zeroes
#series = df.level_scaled.values
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler(feature_range=(0.01, 1))

series = sc.fit_transform(df.level.values.reshape(-1, 1))

split_index = -7*24

#test_series = series [split_index:]
#series = series [:split_index]
#series = series.reshape((len(series), n_features))
#test_series = test_series.reshape((len(test_series), n_features))

series.shape

(78888, 1)

In [107]:
sc_harmonics_list = []
harmonics_list = []
for col in df_harmonics.columns[1:]:
    #print(col)
    sc_harm = MinMaxScaler()
    harmonics_list.append(sc_harm.fit_transform(df_harmonics[col].values.reshape(-1, 1)))
    sc_harmonics_list.append(sc_harm)

In [108]:
sc_rain = MinMaxScaler()
rain = sc_rain.fit_transform(df.rain.values.reshape(-1, 1))

sc_wind = MinMaxScaler()
vel_wind = sc_wind.fit_transform(df.vel_wind.values.reshape(-1, 1))

sc_dir_wind = MinMaxScaler()
dir_wind = sc_dir_wind.fit_transform(df.dir_wind.values.reshape(-1, 1))

sc_lunar = MinMaxScaler()
lunar = sc_lunar.fit_transform(df.inv_dist.values.reshape(-1, 1))


In [109]:
harmonics_list.extend([series, rain, vel_wind, dir_wind])

len(harmonics_list),#harmonics_list

(12,)

In [110]:
input_series = np.hstack(harmonics_list)
input_series.shape

(78888, 12)

In [111]:
# 24h ahead single preds
steps_ahead = 168-1
padded_series = np.expand_dims(np.concatenate([series.flatten(), np.zeros(steps_ahead)]), axis = -1)
padded_series

target_series = np.roll(padded_series, -steps_ahead)[:series.shape[0]] #need to discard the last as well

In [112]:
target_series = target_series[:-steps_ahead]  #removing the zero padding
series = series[:-steps_ahead] #removing the zero padding
input_series = input_series[:-steps_ahead]
series.shape, target_series.shape, input_series.shape

((78721, 1), (78721, 1), (78721, 12))

In [129]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, CuDNNLSTM, LeakyReLU, Dropout, BatchNormalization
from keras.utils import multi_gpu_model
from keras import backend as K
from keras.optimizers import Adam 


# define model
def make_model(n_input, n_features, 
               verbose = False, multi = True, use_CuDNNLSTM = True,
              loss = "mse", metrics = ["mae", "mape"], opt = "adam",
              lr = 0.001, batch_size = 512, reg = True):
    K.clear_session()
    LSTM_layer = LSTM if not use_CuDNNLSTM else CuDNNLSTM
    if opt == "adam":
        opt = Adam (lr = lr )
    else:
        opt = opt
    
    model = Sequential()
    
    model.add(LSTM_layer(512, input_shape=(n_input, n_features), return_sequences=True))
    if reg:
        model.add(BatchNormalization())
    model.add(LeakyReLU())
    if reg:
        model.add(Dropout(rate = 0.5))
    for i in range(0):
        model.add(LSTM_layer(512, return_sequences=True))
        if reg:
            model.add(BatchNormalization())
        model.add(LeakyReLU())
        if reg:
            model.add(Dropout( rate = 0.3))
    
    for i in range(1):
        model.add(LSTM_layer(256, return_sequences = False))
        if reg:
            model.add(BatchNormalization())
        model.add(LeakyReLU())
        if reg:
            model.add(Dropout(rate = 0.4))

    model.add(Dense(128))
    model.add(LeakyReLU())
    if reg:
        model.add(Dropout(rate = 0.3))
    
    model.add(Dense(1))
    
    model.compile(optimizer=opt, loss=loss, metrics = metrics)
    if verbose:
        model.summary()
    if multi:
        mmodel = multi_gpu_model(model, 2)
        mmodel.compile(optimizer=opt, loss=loss, metrics = metrics)
        return mmodel, model
    else: 
        
        return None, model

In [114]:
n_input = 1*12

train_generator = TimeseriesGenerator(input_series, target_series, start_index=0, end_index=len(series) + split_index-1, length=n_input, batch_size=1)
#val_generator = TimeseriesGenerator(series, target_series, start_index=len(series) + 2*split_index-n_input, end_index=len(series)+split_index-1, length=n_input, batch_size=1)
test_generator = TimeseriesGenerator(input_series, target_series, start_index=len(series) + split_index -n_input, end_index=len(series)-1, length=n_input, batch_size=1)

In [115]:
len(train_generator), len(test_generator)

(78541, 168)

In [116]:
train_list = []
test_list = []
for train,test in train_generator:
    train_list.append(train)
    test_list.append(test)
train_input = np.concatenate(train_list, axis = 0)
train_targets = np.concatenate(test_list, axis = 0)
train_input.shape, train_targets.shape

((78541, 12, 12), (78541, 1))

In [117]:
train_list = []
test_list = []
for train, test in test_generator: #some weird but with the last access
    train_list.append(train)
    test_list.append(test)

    
test_input = np.concatenate(train_list, axis = 0)
test_targets = np.concatenate(test_list, axis = 0)
test_input.shape, test_targets.shape

((168, 12, 12), (168, 1))

In [118]:
import wandb
wandb.login()


True

In [131]:
from wandb.keras import WandbCallback as wb

wandb.init(project = "venice_is_drowing", name = f"lstm_single_output_test_168h_{n_input}_input_{input_series.shape[1]}_feat")
cfg = wandb.config
batch_size = 512
cfg.batch_size = batch_size
cfg.n_features = n_features
cfg.n_input = n_input
cfg.epochs = 250

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Wandb version 0.8.22 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [130]:
mmodel, model = make_model(n_input, input_series.shape[1], opt = "adam", lr = 0.001, reg = True, 
                           multi = True, use_CuDNNLSTM = True, verbose = True,
                           loss = "mape", metrics = ["mse","mae", "mape"])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_1 (CuDNNLSTM)     (None, 12, 512)           1077248   
_________________________________________________________________
batch_normalization_1 (Batch (None, 12, 512)           2048      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 12, 512)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 512)           0         
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (None, 256)               788480    
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
__________

In [138]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from myutils import TrainingPlot


plot_losses = TrainingPlot()
es = EarlyStopping(monitor = "val_loss", patience = 50, restore_best_weights = True, verbose = 1)
rl = ReduceLROnPlateau(monitor = "val_loss", patience = 25, cooldown=10, factor=0.5, verbose = 1)

# fit model
history = mmodel.fit(
    train_input, train_targets,
    epochs=cfg.epochs,
    batch_size = cfg.batch_size, 
    
    verbose=1, 
    shuffle = False,
    
    #validation_data=ttest_generator, 
    #validation_steps = 6,
    validation_data = (test_input, test_targets),
    
    callbacks=[plot_losses, 
              es, rl,
              wb(log_gradients=False, log_weights=False)
              ]
)

Train on 78541 samples, validate on 168 samples
Epoch 1/250
  512/78541 [..............................] - ETA: 7s - loss: 16.1357 - mean_squared_error: 0.0101 - mean_absolute_error: 0.0783 - mean_absolute_percentage_error: 16.1357

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Wandb version 0.8.22 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


 2048/78541 [..............................] - ETA: 1:12 - loss: 18.7315 - mean_squared_error: 0.0113 - mean_absolute_error: 0.0858 - mean_absolute_percentage_error: 18.7315

/home/newuser/.local/lib/python3.6/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (1.698995). Check your callbacks.
  % delta_t_median)
/home/newuser/.local/lib/python3.6/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.850842). Check your callbacks.
  % delta_t_median)


KeyboardInterrupt: 

In [134]:
model.evaluate(test_input,test_targets)

168/168 [==============================] - 0s 2ms/step


[9.123519443330311,
 0.0013734966383448669,
 0.030145931988954544,
 9.123519443330311]

In [139]:
def calc_mape(real, pred):
    return (100*np.abs(real-pred)/real/real.shape[0]).sum()
print(calc_mape(real_data, real_preds)) #mape on real-data preds
#print(calc_mape(real_data, preds)) #mape on iterated preds #not really makes sense with 168 steps ahead
print(calc_mape(real_data[0:1], real_preds[0:1]))

9.12351936734321
9.03350650272615


In [142]:
from sklearn.metrics import mean_squared_error
from math import sqrt


rmse_real_preds = sqrt(mean_squared_error(sc.inverse_transform(real_preds.reshape(-1, 1)), sc.inverse_transform(real_data.reshape(-1, 1))))
#rmse_preds = sqrt(mean_squared_error(sc.inverse_transform(preds1.reshape(-1, 1)), sc.inverse_transform(real_data.reshape(-1, 1))))
rmse_single_pred = sqrt(mean_squared_error(sc.inverse_transform(real_data[0:1].reshape(-1, 1)), sc.inverse_transform(real_preds[0:1].reshape(-1, 1))))

rmse_real_preds,  rmse_single_pred

(8.235714803065974, 10.860282897949205)